In [ ]:
import pandas as pd
df=pd.read_csv('/content/breastcancer (1).csv')
df.head

<bound method NDFrame.head of            id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0      842302         M        17.99         10.38          122.80     1001.0   
1      842517         M        20.57         17.77          132.90     1326.0   
2    84300903         M        19.69         21.25          130.00     1203.0   
3    84348301         M        11.42         20.38           77.58      386.1   
4    84358402         M        20.29         14.34          135.10     1297.0   
..        ...       ...          ...           ...             ...        ...   
564    926424         M        21.56         22.39          142.00     1479.0   
565    926682         M        20.13         28.25          131.20     1261.0   
566    926954         M        16.60         28.08          108.30      858.1   
567    927241         M        20.60         29.33          140.10     1265.0   
568     92751         B         7.76         24.54           47.92      181.0  

In [ ]:
x=df.iloc[:,~df.columns.isin(['diagnosis'])].values
y=df.iloc[:, 1].values
print(x)
print(y)

[[8.4230200e+05 1.7990000e+01 1.0380000e+01 ... 4.6010000e-01
  1.1890000e-01           nan]
 [8.4251700e+05 2.0570000e+01 1.7770000e+01 ... 2.7500000e-01
  8.9020000e-02           nan]
 [8.4300903e+07 1.9690000e+01 2.1250000e+01 ... 3.6130000e-01
  8.7580000e-02           nan]
 ...
 [9.2695400e+05 1.6600000e+01 2.8080000e+01 ... 2.2180000e-01
  7.8200000e-02           nan]
 [9.2724100e+05 2.0600000e+01 2.9330000e+01 ... 4.0870000e-01
  1.2400000e-01           nan]
 [9.2751000e+04 7.7600000e+00 2.4540000e+01 ... 2.8710000e-01
  7.0390000e-02           nan]]
['M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'B' 'B' 'B' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'B' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M'
 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B'
 'M' 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'M' 'M' 'B' 'B'
 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['diagnosis']= label_encoder.fit_transform(df['diagnosis'])

df['diagnosis'].unique()

array([1, 0])

In [ ]:
class naivebayes:
    def fit(self, x,y):
        n, m = x.shape
        self.classes=np.unique(y)
        numberCLASS=len(self.classes)
        self.MEAN = np.zeros((numberCLASS, m), dtype=np.float64)
        self.VAR = np.zeros((numberCLASS, m), dtype=np.float64)
        self.PRIOR= np.zeros(numberCLASS, dtype=np.float64)

        for indx, c in enumerate (self.classes) :
            xc=x[c == y]
            self.MEAN[indx,:]=xc.mean(axis=0)
            self.VAR[indx,:]=xc.var(axis=0)
            self.PRIOR[indx]=xc.shape[0] / float(n)

    def predict (self, x):
        ypred =[self.predictOneSample(xi) for xi in x]
        return np.array(ypred)

    def predictOneSample (self, xi):
        post=[]
        for indx, c in enumerate(self.classes):
            pr = np.log(self.PRIOR[indx])
            classcond= np.sum(np.log(self.prob_density_fun(indx, xi)))
            pst = pr+classcond
            post.append(pst)
        return self.classes[np.argmax(post)]


    def prob_density_fun(self, classind, xi):
        mean  = self.MEAN[classind]
        var=self.VAR[classind]
        num =np.exp(-((xi - mean)**2) / (2*var))
        denom = np.sqrt(2* np.pi * var)
        p_x_given_y=num /denom
        return p_x_given_y

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets

def accuracy(yt, yp):
    accuracy = np.sum(yt==yp) / len(yt)
    return accuracy

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=0)

nb=naivebayes()
nb.fit(xtrain, ytrain)
pred=nb.predict(xtest)

print("naive bayes accuracy   :   ", accuracy(ytest, pred))

naive bayes accuracy   :    0.6293706293706294


In [ ]:
def f1_score(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    if true_positives == 0:
        return 0
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    if precision == 0 and recall == 0:
        return 0
    f1 = 2 * precision * recall / (precision + recall)
    return f1

f1 = f1_score(ytest, pred)
print(f"F1 score: {f1}")

F1 score: 0


<ipython-input-7-11eda4ea39c4>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  true_positives = sum((y_true == 1) & (y_pred == 1))
<ipython-input-7-11eda4ea39c4>:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  false_positives = sum((y_true == 0) & (y_pred == 1))
<ipython-input-7-11eda4ea39c4>:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  false_negatives = sum((y_true == 1) & (y_pred == 0))
